# <span style="font-size: 1.5em;">Creacion de Base de Datos:</span>
## <span style="font-size: 1.5em;">Buscador de Informes de la pagina web de AGN</span>

#### <span style="font-size: 1.5em;">Librerías</span>

In [1]:
import warnings  # Codigo 1, 
# Librería para controlar y gestionar advertencias.

from requests.packages.urllib3.exceptions import InsecureRequestWarning  # Codigo 1
# Proporciona excepciones específicas para manejar advertencias relacionadas con solicitudes inseguras.

import requests  # Codigo 1, 9
# Librería para realizar solicitudes HTTP y gestionar respuestas.

import pandas as pd  # Codigo 1, 4, 5, 6, 7, 8, 9
# Librería para manipulación y análisis de datos, especialmente para estructuras de datos tabulares.

from IPython.display import display, HTML  # Codigo 1, 8, 10
# Herramientas para mostrar y formatear la salida en entornos Jupyter.

import sqlite3  # Codigo 1, 2, 3, 8, 9, 10
# Librería para trabajar con bases de datos SQLite.

import os  # Codigo 2
# Librería para interactuar con el sistema operativo, como gestionar rutas de archivos.

from sqlite3 import Error  # Codigo 2
# Proporciona una excepción específica para errores relacionados con SQLite.

from typing import List, Tuple  # Codigo 3
# Herramientas para tipado estático en Python.

import time  # Codigo 4, 5, 6, 7, 
# Librería para trabajar con tiempo, como pausas y medición de tiempo.

import pytesseract  # Codigo 9
# Herramienta OCR (Reconocimiento Óptico de Caracteres) para convertir imágenes en texto.

from PIL import Image  # Codigo 9
# Librería para abrir, manipular y guardar diferentes formatos de imágenes.

from pdf2image import convert_from_bytes  # Codigo 9
# Librería para convertir páginas de PDF en imágenes.

from io import BytesIO # Codigo 9
# Herramientas para trabajar con flujos de datos (como archivos en memoria).

from concurrent.futures import ThreadPoolExecutor  # Codigo 9
# Herramienta para ejecución paralela y concurrente.


#### <span style="font-size: 1.5em;">Funciones</span>

In [2]:
# Codigo 1
# 1. Función para suprimir advertencias
def suprimir_advertencias():
    # Ignora las advertencias de tipo InsecureRequestWarning
    warnings.simplefilter('ignore', InsecureRequestWarning)

# 2. Función para hacer solicitudes a la API
def solicitar_api(url, intentos=5, tiempo_espera=60):
    # Intenta hacer la solicitud a la API un número determinado de veces
    for _ in range(intentos):
        try:
            # Realiza una solicitud GET a la URL proporcionada
            response = requests.get(url, verify=False, timeout=tiempo_espera)
            return response
        except requests.exceptions.Timeout:
            # Si hay un error de tiempo de espera, intenta nuevamente
            continue
    # Si todos los intentos fallan, retorna None
    return None

# 3. Función para estilizar y mostrar DataFrames
def mostrar_dataframe_estilizado(df, num_head=None, num_tail=None):
    
    # Si num_head y num_tail son None, mostrar todo el DataFrame
    if num_head is None and num_tail is None:
        df_resumen = df
    else:
        # Si no, concatenar las filas del principio y del final según los valores de num_head y num_tail
        df_resumen = pd.concat([df.head(num_head), df.tail(num_tail)])
    
    styled_table = df_resumen.style.set_table_styles([
        {'selector': 'table',
         'props': [('border', '2px solid black'), ('border-collapse', 'collapse')]},
        {'selector': 'th',
         'props': [('background-color', 'lightgray'), ('border', '1px solid black'), ('padding', '5px')]},
        {'selector': 'td',
         'props': [('border', '1px solid black'), ('padding', '5px')]}
    ])
    
    # Mostrar el DataFrame estilizado
    display(styled_table)


# 4. Función para guardar el DataFrame en la base de datos SQLite, reemplazando los datos existentes. 
    """
    solo para Tablas "detalle_Informes", "codigo_archivos", "codigo_tid" y "codigo_infografias"
    Parámetros:
    - df: DataFrame que se quiere guardar.
    - tabla: Nombre de la tabla en la base de datos donde se guardará el DataFrame.
    - db_ubicacion: Ruta de la base de datos SQLite. Por defecto es 'informes_agn.db'.

    Retorna:
    - None
    """

def guardar_en_sqlite(df, tabla, db_ubicacion='informes_agn.db'):
    # Establece una conexión con la base de datos SQLite
    conn = sqlite3.connect(db_ubicacion)

    # Crea un cursor para ejecutar comandos SQL
    cur = conn.cursor()

    try:
        # Guarda el DataFrame en la base de datos, reemplazando los datos existentes
        df.to_sql(tabla, conn, if_exists='replace', index=False)
    except sqlite3.IntegrityError:
        # Si hay un error de integridad (por ejemplo, datos duplicados), muestra un mensaje y continúa
        print("Se encontró un error de integridad al reemplazar los datos.")

    # Cierra la conexión a la base de datos
    conn.close()

#### <span style="font-size: 1.5em;">Base de datos SQLite</span>

SQLite se destaca en ciencia de datos para análisis iniciales y prototipados, gracias a su naturaleza sencilla y adaptabilidad.  Se seleccionó para este proyecto por los siguientes motivos:

- Portabilidad: Reside en un solo archivo, lo que simplifica su movilidad.
- Practicidad: Está libre de configuraciones y es perfecto para desarrollos ágiles.
- Eficiencia: Es óptimo para manejar datos de tamaño pequeño a mediano.
- Integración: Ofrece extenso soporte para lenguajes, incluido Python.
- Economía: Es una solución de código abierto y sin coste.
- Versatilidad: Es compatible con múltiples sistemas operativos.

    BD:
        
        "informes_agn.db"
        
    Tablas:
        
        "detalle_Informes"
        
        "codigo_archivos"
        
        "codigo_tid"
        
        "codigo_infografias"
        
        "resolucion_informe_texto"

In [3]:
# Codigo 2
# Función para crear una conexión con la base de datos SQLite.
def create_connection(db_file):
    conn = None
    try:
        # Intenta establecer una conexión con el archivo de base de datos.
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        # Imprime cualquier error que ocurra durante la conexión.
        print(e)

# Función para crear una tabla en la base de datos
def create_table(conn, create_table_sql):
    try:
        # Crea un cursor para ejecutar comandos SQL.
        c = conn.cursor()
        # Ejecuta el comando SQL para crear la tabla.
        c.execute(create_table_sql)
    except Error as e:
        # Imprime cualquier error que ocurra durante la creación de la tabla.
        print(e)

def main():
    # Obtener la ubicación actual y construir la ruta al archivo de base de datos
    db_location = os.path.join(os.getcwd(), 'informes_agn.db')

    # Definición de las tablas
    sql_create_detalle_Informes_table = """
        CREATE TABLE IF NOT EXISTS detalle_Informes (
            codigo_nid integer PRIMARY KEY,
            titulo text NOT NULL,
            descripcion text,
            web text,
            actuacion text,
            ano integer,
            periodo_start DATE,
            periodo_end DATE,
            numero_resolucion integer,
            auditoria_coordinada text,
            titulo_difusion text,
            archivo_difusion text,
            cod_tid_tipo_de_auditoria text,
            cod_tid_organismo_auditado text,
            cod_tid_ods text,
            cod_tid_sector_publico_nacional text,
            cod_tid_objeto_de_auditoria text,
            cod_tid_jurisdiccion text,
            cod_tid_tags text,
            cod_fid_anexo text,
            cod_fid_ficha text,
            cod_fid_imagen text,
            cod_fid_informe integer,
            cod_fid_informe_en_video text,
            cod_fid_informe_multimedia text,
            cod_fid_resolucion integer,
            infografia text
        );
    """
    sql_create_codigo_archivos_table = """
        CREATE TABLE IF NOT EXISTS codigo_archivos (
            cod_fid integer PRIMARY KEY,
            file_name text NOT NULL,
            url text
        );
    """
    sql_create_codigo_tid_table = """
        CREATE TABLE IF NOT EXISTS codigo_tid (
            cod_tid integer PRIMARY KEY,
            base_url text,
            cod_tid_contenido text
        );
    """
    sql_create_codigo_infografias_table = """
        CREATE TABLE IF NOT EXISTS codigo_infografias (
            cod_infografias integer PRIMARY KEY,
            title text NOT NULL,
            nombre_archivo text,
            cod_archivo text
        );
    """
    sql_create_resolucion_informe_texto_table = """
        CREATE TABLE IF NOT EXISTS resolucion_informe_texto (
            codigo_nid integer PRIMARY KEY,
            url_resolucion text,
            texto_resolucion text,
            url_informe text,
            texto_informe text
        );
    """

    # Crea una conexión a la base de datos
    conn = create_connection(db_location)

    # Crea las tablas
    if conn is not None:
        create_table(conn, sql_create_detalle_Informes_table)
        create_table(conn, sql_create_codigo_archivos_table)
        create_table(conn, sql_create_codigo_tid_table)
        create_table(conn, sql_create_codigo_infografias_table)
        create_table(conn, sql_create_resolucion_informe_texto_table)
        conn.close()  # Cierra la conexión después de crear las tablas
    else:
        # Imprime un mensaje de error si no se puede establecer la conexión.
        print("Error! No se puede crear la conexión a la base de datos.")

# Si este archivo se ejecuta como un script, llama a la función main
if __name__ == '__main__':
    main()

Ver datos en la base

In [18]:
# Codigo 3
# Establecemos la ubicación de la base de datos
DB_LOCATION = "informes_agn.db"

# Definimos una función para obtener todos los nombres de las tablas en la base de datos
def get_all_tables(conn: sqlite3.Connection) -> List[str]:
    # Ejecutamos una consulta SQL para obtener los nombres de todas las tablas
    cursor = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    # Extraemos los nombres de las tablas de los resultados y los retornamos en una lista
    tables = [table[0] for table in cursor.fetchall()]
    return tables

# Definimos una función para obtener información sobre las columnas de una tabla específica
def get_column_info(conn: sqlite3.Connection, table_name: str) -> List[Tuple[str, int]]:
    # Ejecutamos una consulta SQL para obtener detalles de las columnas de la tabla
    cursor = conn.execute(f"PRAGMA table_info(\"{table_name}\")")
    # Extraemos el nombre y el tipo de cada columna y los retornamos en una lista de tuplas
    columns_info = [(col[1], col[2]) for col in cursor.fetchall()]
    return columns_info

# Definimos una función para contar los valores no nulos en una columna específica de una tabla
def count_non_null_values(conn: sqlite3.Connection, table_name: str, column_name: str) -> int:
    # Ejecutamos una consulta SQL para contar los valores no nulos de la columna especificada
    cursor = conn.execute(f"SELECT COUNT(\"{column_name}\") FROM \"{table_name}\" WHERE \"{column_name}\" IS NOT NULL")
    # Extraemos el conteo de los resultados y lo retornamos
    count = cursor.fetchone()[0]
    return count

# Definimos la función principal que ejecutará las funciones anteriores y mostrará la información
def main():
    # Creamos una conexión a la base de datos
    conn = sqlite3.connect(DB_LOCATION)
    
    # Verificamos si la conexión fue exitosa
    if not conn:
        print("Error! No se puede crear la conexión a la base de datos.")
        return
    
    # Obtenemos la lista de tablas en la base de datos
    tables = get_all_tables(conn)
    
    # Iteramos sobre cada tabla para mostrar su información
    for table_name in tables:
        print(f"Tabla: {table_name}")
        
        # Obtenemos información sobre las columnas de la tabla
        columns_info = get_column_info(conn, table_name)
        print(f"Cantidad de columnas: {len(columns_info)}")

        # Iteramos sobre cada columna para mostrar su información
        for column_name, column_type in columns_info:
            count = count_non_null_values(conn, table_name, column_name)
            print(f"Columna: '{column_name}' (Tipo: {column_type}), Cantidad de datos: {count}")

        print("----")
    
    # Cerramos la conexión a la base de datos
    conn.close()

# Si este archivo se ejecuta directamente, llamamos a la función main
if __name__ == '__main__':
    main()

Tabla: resolucion_informe_texto
Cantidad de columnas: 5
Columna: 'codigo_nid' (Tipo: INTEGER), Cantidad de datos: 4349
Columna: 'url_resolucion' (Tipo: TEXT), Cantidad de datos: 4349
Columna: 'texto_resolucion' (Tipo: TEXT), Cantidad de datos: 4349
Columna: 'url_informe' (Tipo: TEXT), Cantidad de datos: 3949
Columna: 'texto_informe' (Tipo: TEXT), Cantidad de datos: 4349
----
Tabla: detalle_Informes
Cantidad de columnas: 27
Columna: 'codigo_nid' (Tipo: INTEGER), Cantidad de datos: 4347
Columna: 'titulo' (Tipo: TEXT), Cantidad de datos: 4347
Columna: 'descripcion' (Tipo: TEXT), Cantidad de datos: 791
Columna: 'web' (Tipo: TEXT), Cantidad de datos: 4347
Columna: 'actuacion' (Tipo: TEXT), Cantidad de datos: 3937
Columna: 'ano' (Tipo: INTEGER), Cantidad de datos: 4347
Columna: 'periodo_start' (Tipo: TEXT), Cantidad de datos: 3709
Columna: 'periodo_end' (Tipo: TEXT), Cantidad de datos: 3709
Columna: 'numero_resolucion' (Tipo: REAL), Cantidad de datos: 4316
Columna: 'auditoria_coordinada' (Ti

#### <span style="font-size: 1.5em;">Tabla 1 - "detalle_Informes"</span>

Extracción de detalle descriptivos (atributos), así como los códigos de los datos empleados en los filtros del sitio web (cod_tid_) y códigos de descarga de archivos anexados (cod_fid_).

In [5]:
# Codigo 4

# Función para extraer relaciones.
def extract_relationship(relationship):
    # Si la relación es una lista, extrae todos los IDs y los une con comas
    if isinstance(relationship, list):
        return ', '.join([str(rel.get('meta', {}).get('drupal_internal__target_id', None)) for rel in relationship])
    # Si la relación es un diccionario, extrae el ID
    elif isinstance(relationship, dict):
        return str(relationship.get('meta', {}).get('drupal_internal__target_id', None))
    # Si no es ni lista ni diccionario, retorna None
    else:
        return None

# Código principal
# Definir la URL de la API
url = 'https://webagnapi.agn.gob.ar/api/node/informes'
data_list = []

# Suprimir advertencias
suprimir_advertencias()

# Bucle para recorrer todas las páginas de la API
while url:
    # Hacer la solicitud a la API
    response = solicitar_api(url, intentos=5, tiempo_espera=10)
    
    # Verificar si la respuesta es None (en caso de que todos los intentos fallen)
    if response is None:
        print("No se pudo obtener una respuesta de la API después de varios intentos.")
        break

    # Convertir la respuesta en formato JSON
    data = response.json()

    # Extraer la información requerida y guardarla en una lista
    for item in data['data']:
        attributes = item['attributes']
        relationships = item['relationships']

        # Extraer los atributos
        codigo_nid = attributes.get('drupal_internal__nid', None)
        titulo = attributes.get('titulo', None)
        descripcion = attributes.get('cuerpo', {}).get('value', None) if attributes.get('cuerpo') else None   
        web = attributes.get('alias', None)
        actuacion = attributes.get('actuacion', {}).get('value', None) if attributes.get('actuacion') else None
        ano = attributes.get('ano', None)
        periodo_start = attributes.get('periodo_auditado', {}).get('value', None) if attributes.get('periodo_auditado') else None
        periodo_end = attributes.get('periodo_auditado', {}).get('end_value', None) if attributes.get('periodo_auditado') else None
        numero_resolucion = attributes.get('resolucion', None)
        auditoria_coordinada = attributes.get('auditoria_coordinada', None)
        titulo_difusion = attributes.get('titulo_difusion', None)
        archivo_difusion = attributes.get('archivo_difusion', None)

        # Extraer relaciones usando la función definida previamente
        """Tabla 2 - "codigo_tid" - En esta lista se listan los codigos relacionados al contenido en la base "codigo_tid" """
        cod_tid_tipo_de_auditoria = extract_relationship(relationships.get('tipo_de_auditoria', {}).get('data', {}))
        cod_tid_organismo_auditado = extract_relationship(relationships.get('organismo_auditado', {}).get('data', {}))
        cod_tid_ods = extract_relationship(relationships.get('ods', {}).get('data', {}))
        cod_tid_sector_publico_nacional = extract_relationship(relationships.get('sector_publico_nacional', {}).get('data', {}))
        cod_tid_objeto_de_auditoria = extract_relationship(relationships.get('objeto_de_auditoria', {}).get('data', {}))
        cod_tid_jurisdiccion = extract_relationship(relationships.get('jurisdiccion', {}).get('data', {}))
        cod_tid_tags = extract_relationship(relationships.get('palabras_claves', {}).get('data', {}))
        """Tabla 3 - "codigo_archivos" - En esta lista se listan los codigos relacionados al listado de archivos en la 
        base "codigo_archivos" """
        cod_fid_anexo = extract_relationship(relationships.get('anexo', {}).get('data', {}))
        cod_fid_ficha = extract_relationship(relationships.get('ficha', {}).get('data', {}))
        cod_fid_imagen = extract_relationship(relationships.get('imagen', {}).get('data', {}))
        cod_fid_informe = extract_relationship(relationships.get('informe', {}).get('data', {}))
        cod_fid_informe_en_video = extract_relationship(relationships.get('informe_en_video', {}).get('data', {}))
        cod_fid_informe_multimedia = extract_relationship(relationships.get('informe_multimedia', {}).get('data', {}))
        cod_fid_resolucion = extract_relationship(relationships.get('resolucion_archivo', {}).get('data', {}))
        """Tabla 4 - "codigo_infografias" - En esta lista se listan los codigos relacionados al listado de archivos en la 
        base "codigo_infografias" """     
        infografia = extract_relationship(relationships.get('infografia', {}).get('data', {}))
    
        
        data_list.append([codigo_nid, titulo, descripcion, web, actuacion, ano, periodo_start, periodo_end, numero_resolucion, auditoria_coordinada, titulo_difusion, archivo_difusion, cod_tid_tipo_de_auditoria, cod_tid_organismo_auditado, cod_tid_ods, cod_tid_sector_publico_nacional, cod_tid_objeto_de_auditoria, cod_tid_jurisdiccion, cod_tid_tags, cod_fid_anexo, cod_fid_ficha, cod_fid_imagen, cod_fid_informe, cod_fid_informe_en_video, cod_fid_informe_multimedia, cod_fid_resolucion, infografia])

    # Obtener el enlace a la siguiente página de resultados, si existe
    url = data['links']['next']['href'] if 'next' in data['links'] else None

    # Espera 1 segundo antes de la próxima solicitud
    time.sleep(1)

# Convertir la lista de datos en un DataFrame
df_detalle_Informes = pd.DataFrame(data_list, columns=['codigo_nid', 'titulo', 'descripcion', 'web', 'actuacion', 'ano', 'periodo_start', 'periodo_end', 'numero_resolucion', 'auditoria_coordinada', 'titulo_difusion', 'archivo_difusion', 'cod_tid_tipo_de_auditoria', 'cod_tid_organismo_auditado', 'cod_tid_ods', 'cod_tid_sector_publico_nacional', 'cod_tid_objeto_de_auditoria', 'cod_tid_jurisdiccion', 'cod_tid_tags', 'cod_fid_anexo', 'cod_fid_ficha', 'cod_fid_imagen', 'cod_fid_informe', 'cod_fid_informe_en_video', 'cod_fid_informe_multimedia', 'cod_fid_resolucion', 'infografia'])

# Mostrar el DataFrame estilizado
mostrar_dataframe_estilizado(df_detalle_Informes, num_head=3, num_tail=3)

,codigo_nid,titulo,descripcion,web,actuacion,ano,periodo_start,periodo_end,numero_resolucion,auditoria_coordinada,titulo_difusion,archivo_difusion,cod_tid_tipo_de_auditoria,cod_tid_organismo_auditado,cod_tid_ods,cod_tid_sector_publico_nacional,cod_tid_objeto_de_auditoria,cod_tid_jurisdiccion,cod_tid_tags,cod_fid_anexo,cod_fid_ficha,cod_fid_imagen,cod_fid_informe,cod_fid_informe_en_video,cod_fid_informe_multimedia,cod_fid_resolucion,infografia
0,12,ENTIDAD BINACIONAL YACYRETÁ SÍNTESIS DE LOS PRINCIPALES MOTIVOS DE LA ABSTENCIÓN DE OPINIÓN DEL INFORME SOBRE LOS ESTADOS FINANCIEROS POR EL EJERCICIO FINALIZADO EL 31/12/2017,None,/entidad-binacional-yacyreta-sintesis-de-los-principales-motivos-de-la-abstencion-de,400/2018,2019,2017-01-01,2017-12-31,8.000000,None,None,None,29,316,,,,,"27, 28, 36",,None,None,72,,None,71,
1,13,ESTADOS CONTABLES POR EL EJERCICIO FINALIZADO EL 31 DE DICIEMBRE DE 2015 CORRESPONDIENTES A NUCLEOELÉCTRICA ARGENTINA SOCIEDAD ANÓNIMA,None,/estados-contables-por-el-ejercicio-finalizado-el-31-de-diciembre-de-2015-correspondientes,757/2015,2019,2015-01-01,2015-12-31,10.000000,None,None,None,35,888,,,,,"31, 32, 33, 34",,None,None,74,,None,73,
2,14,"EVALUAR LA RENDICIÓN DE CUENTAS DEL USO DEL FONDO ANUAL PREVISTO POR EL ARTÍCULO 4°, INCISO D), DE LA LEY N° 19.108, MODIFICADA POR EL ARTÍCULO 73 DE LA LEY N° 26.215, CORRESPONDIENTE AL EJERCICIO 2017",None,/evaluar-la-rendicion-de-cuentas-del-uso-del-fondo-anual-previsto-por-el-articulo-4deg,494/2018,2019,2017-01-01,2017-12-31,93.000000,None,None,None,,37,,,,,38,,None,None,76,,None,75,
4344,19282,ESTADOS CONTABLES DE CONSTRUCCIÓN DE VIVIENDA PARA LA ARMADA (COVIARA) EJERCICIO 2020.,"En ejercicio de las facultades conferidas por el artículo 118 de la Ley Nº 24.156, la AUDITORÍA GENERAL DE LA NACIÓN examinó los Estados Contables de CONSTRUCCIÓN DE VIVIENDA PARA LA ARMADA (COVIARA) EMPRESA DEL ESTADO que comprenden el Estado de Situación Patrimonial al 31 de diciembre de 2020, el Estado de Resultados, el Estado de Evolución del Patrimonio Neto y el Estado de Flujo de Efectivo correspondientes al ejercicio económico terminado en dicha fecha, así como un resumen de las políticas contables significativas y otra información explicativa incluidas en las notas 1 a 7 y los anexos I a X, los que se firman a los efectos de su identificación. Las cifras y otra información correspondientes al ejercicio económico terminado el 31 de diciembre de 2019 son parte integrante de los Estados Contables mencionados precedentemente y se las presenta con el propósito de que se interpreten exclusivamente en relación con las cifras y con la información del período actual.",/Informe-165-2023,92/2022,2023,2020-01-01,2020-12-31,165.000000,None,None,None,35,406,,,,,,,None,None,17539,,None,17538,
4345,19283,CERTIFICACIÓN DE LAS TRANSFERENCIAS DE FONDOS DEL ESTADO NACIONAL A AEROLÍNEAS ARGENTINAS S.A. PERIODO OCTUBRE 2022,None,/Informe-166-2023,452/2022,2023,2022-01-01,2022-10-31,166.000000,None,None,None,35,555,,,,,"2616, 209",,None,None,17541,,None,17540,
4346,19294,"LEY 26.682, ENTIDADES DE MEDICINA PREPAGA. SUPERINTENDENCIA DE SERVICIOS DE SALUD","Objeto de la auditoría: Superintendencia de Servicios de Salud – auditoría de gestión – Ley 26.682, Entidades de Medicina Prepaga: Análisis del procedimiento para autorizar el incremento de los valores de las cuotas, actividades de control efectuadas y la adecuada protección de los derechos de los usuarios. Periodo auditado 01/01/2017 a 31/12/2019.",/Informe-167-2023,228/2019,2023,2017-01-01,2019-12-31,167.000000,None,None,None,51,1211,,,,,"2999, 80, 3000, 2305",,17586,None,17588,,None,17587,


Guardar información en la base de datos

In [6]:
guardar_en_sqlite(df_detalle_Informes, 'detalle_Informes')

#### <span style="font-size: 1.5em;">Tabla 2 - "codigo_archivos"</span>

Extracción de la información asociada los códigos de descarga de archivos anexados (cod_fid_).

In [7]:
# Codigo 5
# Llamada a la función para suprimir advertencias
suprimir_advertencias()

# Hacer una solicitud GET a la API utilizando la función
url = 'https://webagnapi.agn.gob.ar/api/file/file'
data_list = []

while url:
    response = solicitar_api(url, intentos=5, tiempo_espera=10)
    
    # Si no se obtiene respuesta después de los intentos, se rompe el bucle
    if not response:
        break

    # Convertir la respuesta en JSON
    data = response.json()

    # Extraer la información requerida y guardarla en una lista
    for item in data['data']:
        cod_fid = item['attributes']['drupal_internal__fid']
        file_name = item['attributes']['filename']
        url_file = item['attributes']['uri']['url']
        data_list.append([cod_fid, file_name, url_file])

    # Obtener el enlace a la siguiente página de resultados, si existe
    url = data['links']['next']['href'] if 'next' in data['links'] else None

    # Espera 1 segundo antes de la próxima solicitud
    time.sleep(1)

# Crear un DataFrame a partir de la lista de datos
df_codigo_archivos = pd.DataFrame(data_list, columns=['cod_fid', 'file_name', 'url'])

# Llamada a la función para mostrar el DataFrame estilizado
mostrar_dataframe_estilizado(df_codigo_archivos, num_head=5, num_tail=5)

,cod_fid,file_name,url
0,6,menu.png,/sites/default/files/menu_icons/menu.png
1,7,menu.png,/sites/default/files/menu_icons/menu_0.png
2,8,menu.png,/sites/default/files/menu_icons/menu_1.png
3,20,CTIC.png,/sites/default/files/images/CTIC.png
4,28,organigrama.jpg,/sites/default/files/inline-images/organigrama.jpg
16071,17597,Disposición 211-2023 GAyF.pdf,/sites/default/files/contrataciones/Disposici%C3%B3n%20211-2023%20GAyF.pdf
16072,17598,OC Nº 38-2023 Ediciones RAP S.A..pdf,/sites/default/files/contrataciones/OC%20N%C2%BA%2038-2023%20Ediciones%20RAP%20S.A..pdf
16073,17599,0266554465.jpg,/sites/default/files/informes/0266554465.jpg
16074,17600,Pliego Act. 325-2023 Adquisición de equipamiento médico.pdf,/sites/default/files/contrataciones/Pliego%20Act.%20325-2023%20Adquisici%C3%B3n%20de%20equipamiento%20m%C3%A9dico.pdf
16075,17601,"Pliego Act. 222-2023 Servicio de alquiler, itratamiento del agua.pdf",/sites/default/files/contrataciones/Pliego%20Act.%20222-2023%20Servicio%20de%20alquiler%2C%20itratamiento%20del%20agua.pdf


Guardar información en la base de datos

In [8]:
guardar_en_sqlite(df_codigo_archivos, 'codigo_archivos')

#### <span style="font-size: 1.5em;">Tabla 3 - "codigo_tid"</span>

Extracción de la información asociada los códigos de los datos empleados en los filtros del sitio web (cod_tid_).

In [9]:
# Codigo 6
# Llamada a la función para suprimir advertencias
suprimir_advertencias()

# Lista de URLs
urls = [
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/jurisdiccion',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/objeto_de_auditoria',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/ods',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/organismo_auditado',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/sector_publico_nacional',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/tags',
    'https://webagnapi.agn.gob.ar/api/taxonomy_term/tipo_de_auditoria'
]

data_list = []

# Iterar sobre cada URL
for url in urls:
    while url:
        response = solicitar_api(url, intentos=5, tiempo_espera=10)
        
        # Si no se obtiene respuesta después de los intentos, se rompe el bucle
        if not response:
            break

        # Convertir la respuesta en JSON
        data = response.json()

        # Extraer la información requerida y guardarla en una lista
        for item in data['data']:
            cod_tid = item['attributes']['drupal_internal__tid']
            base_url = item['type'].split('--')[1]  # Descartar "taxonomy_term--"
            cod_tid_contenido = item['attributes'].get('name', item['attributes'].get('nombre'))  # Usar 'name' o 'nombre', lo que esté disponible
            data_list.append([cod_tid, base_url, cod_tid_contenido])

        # Obtener el enlace a la siguiente página de resultados, si existe
        url = data['links']['next']['href'] if 'next' in data['links'] else None

        # Espera 1 segundo antes de la próxima solicitud
        time.sleep(1)

# Crear un DataFrame a partir de la lista de datos
df_codigo_tid = pd.DataFrame(data_list, columns=['cod_tid', 'base_url', 'cod_tid_contenido'])

# Llamada a la función para mostrar el DataFrame estilizado
mostrar_dataframe_estilizado(df_codigo_tid, num_head=5, num_tail=5)


,cod_tid,base_url,cod_tid_contenido
0,297,jurisdiccion,Cuenca
1,298,jurisdiccion,Matanza
2,299,jurisdiccion,Riachuelo
3,291,objeto_de_auditoria,- Estados Contables al 31/12/2017 del Fideicomiso PROICSA del NOA - Componente IV
4,2471,objeto_de_auditoria,contrataciones de significación económica de la Administración Federal de Ingresos Públicos durante el ejercicio 1998
2707,943,tipo_de_auditoria,Documento Técnico
2708,35,tipo_de_auditoria,Estados Contables
2709,29,tipo_de_auditoria,Estados Financieros
2710,51,tipo_de_auditoria,Informe de Gestión
2711,64,tipo_de_auditoria,Informe Especial


Guardar información en la base de datos

In [10]:
guardar_en_sqlite(df_codigo_tid, 'codigo_tid')

#### <span style="font-size: 1.5em;">Tabla 4 - "codigo_infografias"</span>

Extracción de la información asociada los códigos de infografía, se relaciona a la Tabla 1 por “cod_infografias” y a la Tabla 2 por “cod_archivo”.

In [11]:
# Codigo 7
# Llamada a la función para suprimir advertencias
suprimir_advertencias()

# Hacer una solicitud GET a la API
url = 'https://webagnapi.agn.gob.ar/api/node/infografias'
data_list = []

while url:
    response = solicitar_api(url, intentos=5, tiempo_espera=60)
    
    # Si no se obtiene respuesta después de los intentos, se rompe el bucle
    if not response:
        break

    # Convertir la respuesta en JSON
    data = response.json()

    # Extraer la información requerida y guardarla en una lista
    for item in data['data']:
        cod_infografias = item['attributes']['drupal_internal__nid']
        title = item['attributes']['title']
        nombre_archivo_list = []
        cod_archivo_list = []
        for image_data in item['relationships']['field_imagen']['data']:
            nombre_archivo_list.append(image_data['meta']['alt'])
            cod_archivo_list.append(str(image_data['meta']['drupal_internal__target_id']))
        nombre_archivo = ', '.join(nombre_archivo_list)
        cod_archivo = ', '.join(cod_archivo_list)
        data_list.append([cod_infografias, title, nombre_archivo, cod_archivo])

    # Obtener el enlace a la siguiente página de resultados, si existe
    url = data['links']['next']['href'] if 'next' in data['links'] else None

    # Espera 1 segundo antes de la próxima solicitud
    time.sleep(1)

# Crear un DataFrame a partir de la lista de datos
df_codigo_infografias = pd.DataFrame(data_list, columns=['cod_infografias', 'title', 'nombre_archivo', 'cod_archivo'])

# Llamada a la función para mostrar el DataFrame estilizado
mostrar_dataframe_estilizado(df_codigo_infografias, num_head=3, num_tail=3)


,cod_infografias,title,nombre_archivo,cod_archivo
0,690,Gestión Informática - SUBE,"SUBE, Sube 1, Sube 2","5896, 1747, 1748"
1,695,"Informe de seguimiento sobre la Implementación de la Ley 26.639, para la protección de glaciares","Informe de seguimiento sobre la Implementación de la Ley 26.639, para la protección de glaciares",1746
2,696,Declaracion de Buenos Aires,"Declaración de Buenos Aires 1, Declaración de Buenos Aires 1","1749, 1750"
17,12768,Programa 20 - Prevención y Control de Enfermedades Inmuno-prevenibles,"vacunas, vacunas","6031, 6030"
18,14372,Bosques Nativos en la Región Patagónica,"Bosques Nativos en la Región Patagónica - parte 1, Bosques Nativos en la Región Patagónica - parte 2, Bosques Nativos en la Región Patagónica - parte 3","9156, 9154, 9155"
19,17682,LOS CIRCUITOS DE LA PARTICIPACIÓN,"LOS CIRCUITOS DE LA PARTICIPACIÓN, LOS CIRCUITOS DE LA PARTICIPACIÓN","13820, 13821"


Guardar información en la base de datos

In [12]:
guardar_en_sqlite(df_codigo_infografias, 'codigo_infografias')

#### <span style="font-size: 1.5em;">Tabla 5 - "resolucion_informe_texto"</span>

Creación de tabla para su posterior análisis en donde se descarga para cada “código_nid” de la Tabla 1 el archivo pdf de resolución e informe asociado a la Tabla 2.

Crear data frame con los "codigo_nid" nuevos a actualizar y los que indiquen algún error.

In [17]:
# Codigo 8
# Establecer una conexión con la base de datos SQLite llamada 'informes_agn.db'
conn = sqlite3.connect('informes_agn.db')

# Definir la consulta SQL
query = """
-- Subconsulta para identificar errores en la columna 'texto_resolucion'
WITH Errors_resolucion AS (
    SELECT 
        codigo_nid,
        CASE 
            WHEN texto_resolucion LIKE '%Error de descarga%' THEN 'Error de descarga'
            WHEN texto_resolucion LIKE '%Error de conversión%' THEN 'Error de conversión'
        END AS Estado_r
    FROM resolucion_informe_texto
    WHERE 
        texto_resolucion LIKE '%Error de descarga%' OR 
        texto_resolucion LIKE '%Error de conversión%'
),

-- Subconsulta para identificar errores en la columna 'texto_informe'
Errors_informe AS (
    SELECT 
        codigo_nid,
        CASE 
            WHEN texto_informe LIKE '%Error de descarga%' THEN 'Error de descarga'
            WHEN texto_informe LIKE '%Error de conversión%' THEN 'Error de conversión'
        END AS Estado_i
    FROM resolucion_informe_texto
    WHERE 
        texto_informe LIKE '%Error de descarga%' OR 
        texto_informe LIKE '%Error de conversión%'
)

-- Consulta principal
SELECT 
    d.codigo_nid,
    COALESCE(er.Estado_r, 'Nuevo') AS Estado_r,
    COALESCE(r.url, 'Sin resolución') AS url_resolucion,
    COALESCE(ei.Estado_i, 'Nuevo') AS Estado_i,
    COALESCE(i.url, 'Sin informe') AS url_informe
FROM detalle_Informes d
LEFT JOIN codigo_archivos r ON d.cod_fid_resolucion = r.cod_fid
LEFT JOIN codigo_archivos i ON d.cod_fid_informe = i.cod_fid
LEFT JOIN Errors_resolucion er ON d.codigo_nid = er.codigo_nid
LEFT JOIN Errors_informe ei ON d.codigo_nid = ei.codigo_nid
WHERE 
    er.Estado_r IS NOT NULL OR 
    ei.Estado_i IS NOT NULL OR 
    d.codigo_nid NOT IN (SELECT codigo_nid FROM resolucion_informe_texto)
"""

# Ejecutar la consulta SQL y almacenar el resultado en un DataFrame
df_actualizar = pd.read_sql_query(query, conn)

# Cerrar la conexión a la base de datos
conn.close()

# Estilizar el DataFrame para mejorar su presentación visual
mostrar_dataframe_estilizado(df_actualizar)

,codigo_nid,Estado_r,url_resolucion,Estado_i,url_informe
0,12003,Nuevo,/sites/default/files/informes/resolucion_151_2019.pdf,Error de descarga,/sites/default/files/informes/Informe_151_2019.pdf
1,19267,Nuevo,/sites/default/files/informes/2023-154-ResolucionMCI.pdf,Error de descarga,Sin informe


Descarga y procesar un documento (resolución o informe)

In [16]:
# Codigo 9

# Esta función descarga un PDF desde una URL dada
def download_pdf(url):
    try:
        # Realizamos una solicitud GET a la URL
        response = requests.get(url)
        # Verificamos que la respuesta sea exitosa (código 200)
        response.raise_for_status()
        # Devolvemos el contenido del PDF en formato de bytes
        return BytesIO(response.content)
    except requests.RequestException as e:
        # Si hay algún error en la solicitud, devolvemos un mensaje de error
        return f"Error de descarga: {e}"

# Esta función convierte el contenido de un PDF a texto
def pdf_to_text(pdf_content):
    # Si el contenido del PDF es un mensaje de error, lo devolvemos tal cual
    if isinstance(pdf_content, str):
        return pdf_content

    try:
        # Convertimos el contenido del PDF a imágenes
        images = convert_from_bytes(pdf_content.read())
        # Usamos un ThreadPoolExecutor para procesar las imágenes en paralelo
        with ThreadPoolExecutor() as executor:
            # Convertimos cada imagen a texto usando pytesseract
            texts = list(executor.map(pytesseract.image_to_string, images))
        # Devolvemos el texto concatenado y sin espacios al principio o al final
        return "".join(texts).strip()
    except Exception as e:
        # Si hay algún error en la conversión, devolvemos un mensaje de error
        return f"Error de conversión: {e}"

# Esta función procesa una fila del DataFrame y devuelve la información necesaria
def process_row(row):
    # Construimos las URLs completas para la resolución y el informe
    url_resolucion = 'https://www.agn.gob.ar' + str(row['url_resolucion'])
    url_informe = 'https://www.agn.gob.ar' + str(row['url_informe'])

    # Descargamos y convertimos a texto el contenido de la resolución y el informe
    pdf_content_resolucion = download_pdf(url_resolucion)
    texto_resolucion = pdf_to_text(pdf_content_resolucion)
    
    pdf_content_informe = download_pdf(url_informe)
    texto_informe = pdf_to_text(pdf_content_informe)

    # Devolvemos una tupla con el código, las URLs y los textos
    return (row['codigo_nid'], url_resolucion, texto_resolucion, url_informe, texto_informe)

# Función principal
def main():
    # Nos conectamos a la base de datos SQLite
    with sqlite3.connect('informes_agn.db') as conn:
        cursor = conn.cursor()

        # Procesamos cada fila del DataFrame y preparamos los datos para la inserción
        data_to_insert = [process_row(row) for _, row in df_actualizar.iterrows()]

        # Insertamos o actualizamos los datos en la base de datos usando transacciones
        cursor.executemany('''
            INSERT OR REPLACE INTO resolucion_informe_texto
            (codigo_nid, url_resolucion, texto_resolucion, url_informe, texto_informe)
            VALUES (?, ?, ?, ?, ?)
        ''', data_to_insert)

        # Confirmamos los cambios en la base de datos
        conn.commit()

# Si este script se ejecuta como el principal, llamamos a la función main
if __name__ == "__main__":
    main()

Visualizar información (random o por "codigo_nid")

In [ ]:
# Codigo 10

# Función para imprimir en negrita y con un tamaño de fuente más grande
def print_large_bold(text):
    display(HTML(f"<b style='font-size: 20px;'>{text}</b>"))

# Establecer conexión con la base de datos SQLite
conn = sqlite3.connect('informes_agn.db')
c = conn.cursor()

# Comentar la opción que no se va a utilizar

# Opción 1: Consultar una fila al azar de la tabla resolucion_texto
#c.execute('SELECT * FROM resolucion_informe_texto ORDER BY RANDOM() LIMIT 1')

# Opción 2: Consultar una fila de la tabla resolucion_texto basada en el valor de codigo_nid
c.execute('SELECT * FROM resolucion_informe_texto WHERE codigo_nid = ?', (19257,))
row = c.fetchone()

# Mostrar la fila obtenida
if row:
    print_large_bold("codigo_nid:"), print(row[0])
    print_large_bold("url_resolucion:"), print(row[1])
    print_large_bold("texto_resolucion:"), print(row[2])
    print_large_bold("url_Informe:"), print(row[3])
    print_large_bold("texto_Informe:"), print(row[4])
else:
    print("No hay registros en la tabla.")

# Cerrar la conexión a la base de datos
conn.close()